In [77]:
#Imports

#%%capture --no-display
# Dependencies and Setup
import hvplot.pandas
import geopandas as gpd
import pandas as pd
import requests
#import fiona
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns
import datetime as datetime
import time
from scipy.stats import linregress


In [78]:
#The School per zipcode file
# Getting the csv file
school_per_zip_path = "output-data/schools_per_zipcode.csv"
# Read the data
school_per_zip_metadata = pd.read_csv(school_per_zip_path)
school_per_zip_metadata.columns

Index(['Unnamed: 0', 'zip', 'universal-id', 'nces-id', 'state-id', 'name',
       'school-summary', 'type', 'level-codes', 'level', 'street', 'city',
       'state', 'fipscounty', 'phone', 'fax', 'county', 'lat', 'lon',
       'district-name', 'district-id', 'web-site', 'overview-url', 'rating',
       'year', 'rating-description', 'lat_lon'],
      dtype='object')

In [79]:
#The Housing file
# Getting the csv file
housing_data_path = "zillow/zip_avg.csv"
# Read the data
housing_metadata = pd.read_csv(housing_data_path)

# School

In [80]:
#school_per_zip_metadata = school_per_zip_metadata['zip'].astype('int')
school_per_zip_metadata = school_per_zip_metadata.rename(columns ={'zip':"Zipcode"})
school_per_zip_metadata

,Unnamed: 0,Zipcode,universal-id,nces-id,state-id,name,school-summary,type,level-codes,level,...,lat,lon,district-name,district-id,web-site,overview-url,rating,year,rating-description,lat_lon
0,1,30303,1306933,130023104095,7.820119e+10,Graduation Achievement Center Charter High School,Graduation Achievement Center Charter High Sch...,charter,h,"9,10,11,12",...,33.760502,-84.387169,State Charter Schools Provost Academy Georgia,250,gradgeorgia.com,https://www.greatschools.org/georgia/atlanta/6...,NaN,NaN,The GreatSchools Rating helps parents compare ...,"(33.760502, -84.387169)"
1,2,30303,1308742,130228004261,6.601321e+06,Wellspring Living,"Wellspring Living, a public school located in ...",public,"m,h","8,9,11",...,33.754742,-84.385574,Fulton County School District,73,NaN,https://www.greatschools.org/georgia/atlanta/8...,NaN,NaN,The GreatSchools Rating helps parents compare ...,"(33.754742, -84.385574)"
2,3,30303,1309128,A1901569,NaN,The Life School of Atlanta,"The Life School of Atlanta, a private school l...",private,h,"9,10,11,12",...,33.749504,-84.392090,NaN,0,NaN,https://www.greatschools.org/georgia/atlanta/9...,NaN,NaN,The GreatSchools Rating helps parents compare ...,"(33.749504, -84.39209)"
3,5,30334,1306660,NaN,NaN,K12 International Academy,"K12 International Academy , a private school l...",private,"e,m,h","KG,1,2,3,4,5,6,7,8,9,10,11,12",...,33.749733,-84.385696,NaN,0,http://ww2.k12.com/cm/?affl=gr8t&page=sp&schoo...,https://www.greatschools.org/georgia/atlanta/6...,NaN,NaN,The GreatSchools Rating helps parents compare ...,"(33.749733, -84.385696)"
4,6,30334,1306661,NaN,NaN,The Keystone School,"The Keystone School , a private school located...",private,"m,h","6,7,8,9,10,11,12",...,33.749733,-84.385696,NaN,0,http://keystoneschoolonline.com/,https://www.greatschools.org/georgia/atlanta/6...,NaN,NaN,The GreatSchools Rating helps parents compare ...,"(33.749733, -84.385696)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,374,30345,1300770,130174000721,6.443060e+06,Lakeside High School,"Lakeside High School, a public school located ...",public,h,"9,10,11,12",...,33.844547,-84.284828,Dekalb County School District,55,NaN,https://www.greatschools.org/georgia/atlanta/7...,5.0,2022.0,The GreatSchools Rating helps parents compare ...,"(33.844547, -84.284828)"
95,377,30345,1308860,BB160917,NaN,Abm Vietnamese School,"Abm Vietnamese School, a private school locate...",private,"m,h","6,7,8,9,10,11,12",...,33.848419,-84.285812,NaN,0,NaN,https://www.greatschools.org/georgia/atlanta/8...,NaN,NaN,The GreatSchools Rating helps parents compare ...,"(33.848419, -84.285812)"
96,382,30342,1301898,00296991,NaN,Yeshiva Atlanta High School,"Yeshiva Atlanta High School, a private school ...",private,"p,e,m,h","PK,TK,KG,1,2,3,4,5,6,7,8,9,10,11,12",...,33.895885,-84.366486,NaN,0,NaN,https://www.greatschools.org/georgia/atlanta/1...,NaN,NaN,The GreatSchools Rating helps parents compare ...,"(33.895885, -84.366486)"
97,383,30342,1301960,00298387,NaN,The Galloway School,"The Galloway School, a private school located ...",private,"p,e,m,h","PK,TK,KG,1,2,3,4,5,6,7,8,9,10,11,12",...,33.875198,-84.393661,NaN,0,http://www.gallowayschool.org,https://www.greatschools.org/georgia/atlanta/1...,NaN,NaN,The GreatSchools Rating helps parents compare ...,"(33.875198, -84.393661)"


# Housing

In [81]:
#cleaning the housing Data

housing_metadata = housing_metadata.dropna(subset = ["RegionName"])

housing_df = pd.DataFrame({
    "Zipcode":housing_metadata["RegionName"],
    "Average Housing Price":housing_metadata["Average2022"]
})

housing_df["Average Housing Price"] = housing_df["Average Housing Price"].round(decimals = 2)
housing_df["Zipcode"] = housing_df["Zipcode"].astype('int')

housing_df


,Zipcode,Average Housing Price
0,30101,427407.83
1,30102,366971.33
2,30103,266670.33
3,30005,696600.42
4,30009,648391.83
...,...,...
196,30184,369683.83
197,30217,195491.67
198,30182,229584.67
199,30171,328923.42


# Merging

In [82]:
#Merging Data sets

school_vs_housing = pd.merge(
    school_per_zip_metadata,
    housing_df,
    on=['Zipcode', "Zipcode"],
    how = 'inner'
)

school_vs_housing.columns

Index(['Unnamed: 0', 'Zipcode', 'universal-id', 'nces-id', 'state-id', 'name',
       'school-summary', 'type', 'level-codes', 'level', 'street', 'city',
       'state', 'fipscounty', 'phone', 'fax', 'county', 'lat', 'lon',
       'district-name', 'district-id', 'web-site', 'overview-url', 'rating',
       'year', 'rating-description', 'lat_lon', 'Average Housing Price'],
      dtype='object')

# Exporting DataFrame

In [83]:
# Export housing_df
school_vs_housing.to_csv("output-data/school_vs_housing.csv")

# Mapping

In [84]:
map_plot_1 = school_vs_housing.hvplot.points(
    x = "lon",
    y = "lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 750,
    size = "Average Housing Price",
    scale = 0.01,
    xlabel="Longitude",
    ylabel="Latitude",
    title="Housing Prices Vs Schools"
)

# Display the map plot
plt.savefig("output-data/Schools Based on Avg Housing Cost.jpg")
#plt.show()
map_plot_1

/Users/alexrosenberg/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
/Users/alexrosenberg/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
/Users/alexrosenberg/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
/Users/alexrosenberg/opt/anaconda3/li

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Average Housing Price)

<Figure size 640x480 with 0 Axes>